SQL TEST

Number 1

In [ ]:
import pandas as pd

First, we need to read all the data sources and include the names since they are not present in the original files.\
In addition, we include a sample of the data for reviewing it briefly after processing.

In [ ]:
names=['department_id','department']
departments=pd.read_csv("departments.csv",names=names)
departments.head()

,department_id,department
0,1,Product Management
1,2,Sales
2,3,Research and Development
3,4,Business Development
4,5,Engineering


In [ ]:
names=['job_id','job']
jobs=pd.read_csv("jobs.csv",names=names)
jobs.head()

,job_id,job
0,1,Marketing Assistant
1,2,VP Sales
2,3,Biostatistician IV
3,4,Account Representative II
4,5,VP Marketing


In [ ]:
names=['hired_employees_id','name','datetime','department_id','job_id']
hired_employees=pd.read_csv("hired_employees.csv",names=names)
hired_employees.head()

,hired_employees_id,name,datetime,department_id,job_id
0,1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.0
1,2,Ty Hofer,2021-05-30T05:43:46Z,8.0,NaN
2,3,Lyman Hadye,2021-09-01T23:27:38Z,5.0,52.0
3,4,Lotti Crowthe,2021-10-01T13:04:21Z,12.0,71.0
4,5,Gretna Lording,2021-10-10T22:22:17Z,6.0,80.0


Create a 'year' column derived from the 'datetime' column to facilitate filtering information for the year 2021.

In [ ]:
hired_employees['year']=pd.DatetimeIndex(hired_employees['datetime']).year
hired_employees.head()

,hired_employees_id,name,datetime,department_id,job_id,year
0,1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.0,2021.0
1,2,Ty Hofer,2021-05-30T05:43:46Z,8.0,NaN,2021.0
2,3,Lyman Hadye,2021-09-01T23:27:38Z,5.0,52.0,2021.0
3,4,Lotti Crowthe,2021-10-01T13:04:21Z,12.0,71.0,2021.0
4,5,Gretna Lording,2021-10-10T22:22:17Z,6.0,80.0,2021.0


Before filtering the 'hired_employees' DataFrame, we review its shape to analyze the filtered results.

In [ ]:
hired_employees.shape

(1999, 6)

In [ ]:
hired_employees=hired_employees[hired_employees['year']==2021]
hired_employees.shape

(1685, 6)

Afterward, we merge all the DataFrames to create a unified one that can be applied to both questions. We use the 'inner' method due to the data model being a transaction table and two dimensions. For this, we only need the rows that intersect in the join, so it is an inner join

In [ ]:
merged_df = pd.merge(hired_employees, departments, left_on='department_id', right_on='department_id', how='inner')
merged_df = pd.merge(merged_df, jobs, left_on='job_id', right_on='job_id', how='inner')
merged_df.head()

,hired_employees_id,name,datetime,department_id,job_id,year,department,job
0,1,Harold Vogt,2021-11-07T02:48:42Z,2.0,96.0,2021.0,Sales,Health Coach III
1,424,Jeana Costanza,2021-07-31T00:51:12Z,2.0,96.0,2021.0,Sales,Health Coach III
2,873,Aldous Walworche,2021-11-11T15:57:19Z,2.0,96.0,2021.0,Sales,Health Coach III
3,1754,Aldis Pottes,2021-09-07T22:00:56Z,2.0,96.0,2021.0,Sales,Health Coach III
4,482,Emmalynn Robilliard,2021-12-08T16:54:02Z,8.0,96.0,2021.0,Support,Health Coach III


Next is the creation of a column for the 2021 quarter

In [ ]:
merged_df['datetime'] = pd.to_datetime(merged_df['datetime'])
merged_df['quarter'] = merged_df['datetime'].dt.to_period('Q')
merged_df.head()

<ipython-input-13-c14d114f0dfe>:2: UserWarning: Converting to PeriodArray/Index representation will drop timezone information.
  merged_df['quarter'] = merged_df['datetime'].dt.to_period('Q')


,hired_employees_id,name,datetime,department_id,job_id,year,department,job,quarter
0,1,Harold Vogt,2021-11-07 02:48:42+00:00,2.0,96.0,2021.0,Sales,Health Coach III,2021Q4
1,424,Jeana Costanza,2021-07-31 00:51:12+00:00,2.0,96.0,2021.0,Sales,Health Coach III,2021Q3
2,873,Aldous Walworche,2021-11-11 15:57:19+00:00,2.0,96.0,2021.0,Sales,Health Coach III,2021Q4
3,1754,Aldis Pottes,2021-09-07 22:00:56+00:00,2.0,96.0,2021.0,Sales,Health Coach III,2021Q3
4,482,Emmalynn Robilliard,2021-12-08 16:54:02+00:00,8.0,96.0,2021.0,Support,Health Coach III,2021Q4


The next step is to group the table by the 'quarter,' 'job,' and 'department' columns to obtain the number of employees that match within these categories

In [ ]:
result_df = merged_df.groupby(['department', 'job', 'quarter']).size().reset_index(name='number_of_employees')
result_df.head()

,department,job,quarter,number_of_employees
0,Accounting,Account Representative IV,2021Q1,1
1,Accounting,Actuary,2021Q2,1
2,Accounting,Analyst Programmer,2021Q3,1
3,Accounting,Budget/Accounting Analyst III,2021Q2,1
4,Accounting,Cost Accountant,2021Q2,1


Finally, we pivot the table to create columns for each quarter and rename all the columns to ensure correct naming for the intended purpose.

In [ ]:
result_pivot = result_df.pivot_table(index=['department', 'job'], columns='quarter', values='number_of_employees', fill_value=0)
result_pivot.reset_index(inplace=True)
result_pivot.index.name = None
result_pivot.columns.name = None
result_pivot = result_pivot.sort_values(by=['department', 'job'])
result_pivot.columns=['department','job','Q1','Q2','Q3','Q4']
result_pivot.head()


,department,job,Q1,Q2,Q3,Q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0


Number 2

First, we need to obtain a table that contains all the employees per department. With that, we can calculate the mean of this value.

In [ ]:
count_employees = merged_df.groupby('department')['hired_employees_id'].count()
print(count_employees)

department
Accounting                   38
Business Development        187
Engineering                 207
Human Resources             204
Legal                        56
Marketing                   142
Product Management           49
Research and Development    149
Sales                        94
Services                    202
Support                     217
Training                    114
Name: hired_employees_id, dtype: int64


In [ ]:
mean_employees=count_employees.mean()
print(mean_employees)

138.25


After obtaining the mean, we need to create a list of departments that have a total number of employees more than the mean.

In [ ]:
departments_above_mean = merged_df.groupby('department').filter(lambda x: len(x) > mean_employees)['department'].unique()
print(departments_above_mean)

['Support' 'Engineering' 'Human Resources' 'Marketing' 'Services'
 'Research and Development' 'Business Development']


Filter the employee count by department to include only those departments whose total employee count is above the mean.

In [ ]:
count_employees_department = count_employees.loc[departments_above_mean]
print(count_employees_department)

department
Support                     217
Engineering                 207
Human Resources             204
Marketing                   142
Services                    202
Research and Development    149
Business Development        187
Name: hired_employees_id, dtype: int64


Finally, obtain the final result by ordering it based on the count and joining it with the original department table to include the department ID

In [ ]:
result_df = pd.merge(departments, count_employees_department, left_on='department', right_index=True, how='inner')
result_df.columns=['id','department','hired']
result_df=result_df.sort_values(by=['hired'],ascending=False)
print(result_df)

   id                department  hired
7   8                   Support    217
4   5               Engineering    207
5   6           Human Resources    204
6   7                  Services    202
3   4      Business Development    187
2   3  Research and Development    149
8   9                 Marketing    142
